In [14]:

import mysql.connector
import pandas as pd
import plotly.express as px
import geopandas as gpd
from geopandas.tools import geocode
from geopy.geocoders import Nominatim

mydb = mysql.connector.connect(
  host="127.0.0.1",
  port="3306",
  user="root",
  password="cosef56",
  database="306"
)
print(mydb.is_connected())

mycursor = mydb.cursor()
mycursor.execute(
    '''SELECT c.country_name, AVG(c.production_t) AS avg_production
    FROM crops c
    JOIN population p ON c.country_name = p.country_name AND c.country_year =
    p.year
    GROUP BY c.country_name
    HAVING avg_production> 3312915'''
)

df = pd.DataFrame(columns=["Country", "avg_production"])

for a,b in mycursor:
    tmp = pd.DataFrame(
            [{"Country": a, "avg_production": b}])
    df = pd.concat([df, tmp], axis=0, ignore_index=True)          
df["avg_production"] = pd.to_numeric(df["avg_production"])

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

country_df = world[['name', 'iso_a3']]
country_df.columns = ['Country', 'ISO']

merged_df = pd.merge(df, country_df, on='Country', how='left')

iso_code_updates = {
    'Serbia and Montenegro': 'SRB',  # Update with the correct ISO code
    'United States': 'USA',  # Update with the correct ISO code
    'USSR': 'RUS',  # Update with the correct ISO code
    'Yugoslavia': 'YUG'  # Update with the correct ISO code
}

# Update the ISO codes in the DataFrame
for country, iso_code in iso_code_updates.items():
    merged_df.loc[merged_df['Country'] == country, 'ISO'] = iso_code
# Drop the original 'Country' column

sorted_df = merged_df.sort_values('avg_production', ascending=False)

# Get the top 10 countries with the highest 'avg_production' values
top_10_countries = sorted_df.head(10)
print(top_10_countries)

# Print the top 10 countries





True
          Country  avg_production  ISO
21  United States    2.154362e+08  USA
3           China    1.038803e+08  CHN
1          Brazil    3.377674e+07  BRA
11         Mexico    1.540916e+07  MEX
0       Argentina    1.481288e+07  ARG
20        Ukraine    1.271751e+07  UKR
8           India    1.147214e+07  IND
22           USSR    1.138916e+07  RUS
6          France    1.129847e+07  FRA
14        Romania    9.207796e+06  ROU


C:\Users\yusuf\AppData\Local\Temp\ipykernel_26996\1325804298.py:35: FutureWarning:

The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.



In [19]:
fig = px.scatter_geo(top_10_countries, locations="ISO",
                     color="Country", # which column to use to set the color of markers
                     hover_name="Country", # column added to hover information
                     size="avg_production", # size of markers
                     projection="natural earth",
                     size_max=35)
fig.show()